# YOLOv8 Preparation

The content of this notebook can be found in the file `prepare_yolo.py`. This notebook is only for understanding the code.

In [1]:
import os
from os.path import join, basename
from glob import glob
from importlib import reload
import time

import matplotlib.pyplot as plt
import numpy as np
from natsort import natsorted
from PIL import Image
import torch
from torch.utils.data import DataLoader
from ultralytics import YOLO
import yaml
import shutil
from sklearn.model_selection import train_test_split

import dataset
reload(dataset)
from dataset import DotaDataset

#### Project Parameters

In [2]:
DATA_DIR = "../04-Data"

#### Locate images and targets

In [3]:
train_targets_path = os.path.join(DATA_DIR,"patches_1024", 'train_targets_patches')
train_images_path = os.path.join(DATA_DIR,"patches_1024",'train_images_patches')

train_images_list = glob(join(train_images_path, "**", "*.png"),recursive=True)
train_images_list = natsorted(train_images_list, key=lambda y: y.lower())
train_targets_list = glob(join(train_targets_path, "**", "*.txt"),recursive=True)
train_targets_list = natsorted(train_targets_list, key=lambda y: y.lower())

val_targets_path = os.path.join(DATA_DIR,"patches_1024",'val_targets_patches')
val_images_path = os.path.join(DATA_DIR,'patches_1024', "val_images_patches")

val_images_list = glob(join(val_images_path, "**", "*.png"),recursive=True)
val_images_list = natsorted(val_images_list, key=lambda y: y.lower())
val_targets_list = glob(join(val_targets_path, "**", "*.txt"),recursive=True)
val_targets_list = natsorted(val_targets_list, key=lambda y: y.lower())

test_targets_path = os.path.join(DATA_DIR,"patches_1024",'test_targets_patches')
test_images_path = os.path.join(DATA_DIR,'patches_1024', "test_images_patches")

test_images_list = glob(join(test_images_path, "**", "*.png"),recursive=True)
test_images_list = natsorted(test_images_list, key=lambda y: y.lower())
test_targets_list = glob(join(test_targets_path, "**", "*.txt"),recursive=True)
test_targets_list = natsorted(test_targets_list, key=lambda y: y.lower())

In [4]:
print(f"Train images: {len(train_images_list)}")
print(f"Train targets: {len(train_targets_list)}")
print(f"Val images: {len(val_images_list)}")
print(f"Val targets: {len(val_targets_list)}")
print(f"Test images: {len(test_images_list)}")
print(f"Test targets: {len(test_targets_list)}")

Train images: 21132
Train targets: 21132
Val images: 3139
Val targets: 3139
Test images: 2279
Test targets: 2279


#### Modify dataset to YOLO standard

YOLO models require the bounding boxes to be in the format ```label x_center y_center width height```and they need a specific data directory structure

In [5]:
def extract_hbb(pxl_coordinates: list):
    '''
    Creates a dictionary of bounding box information and returns a list of dictionaries containing bounding box information

    pxl_coordinates: List of strings
    '''
    bounding_boxes=[]
    for location in pxl_coordinates: 
        parts = location.strip().split()
        bounding_boxes.append({'label': parts[-2],
                              'x1': float(parts[0]), # x_min
                              'y1': float(parts[1]), # y_min
                              'x2': float(parts[2]), 
                              'y2': float(parts[3]), 
                              'x3': float(parts[4]), # x_max
                              'y3': float(parts[5]), # y_max
                              'x4': float(parts[6]), 
                              'y4': float(parts[7]), 
                              'difficulty': int(parts[-1])})
    return bounding_boxes

In [6]:
def read_txt(path):
    '''
    Reads .txt files and returns a list of strings (lines of the .txt file)

    path: Path to .txt file
    '''
    with open(path, 'r') as file:
        lines = file.readlines()
    return lines

In [7]:
class_to_idx = {
           "ship" : 0, 
           "storage-tank" : 1, 
           "baseball-diamond" : 2, 
           "tennis-court" : 3, 
           "basketball-court" : 4, 
           "ground-track-field" : 5, 
           "bridge" : 6, 
           "large-vehicle" : 7, 
           "small-vehicle" : 8, 
           "helicopter" : 9, 
           "swimming-pool" : 10, 
           "roundabout" : 11, 
           "soccer-ball-field" : 12, 
           "plane" : 13, 
           "harbor" : 14,
           "container-crane" : 15,
           "no_object": 16} # for empty images

In [8]:
idx_to_class = {
           0 : "ship", 
           1 : "storage-tank", 
           2 : "baseball-diamond", 
           3 : "tennis-court", 
           4 : "basketball-court", 
           5 : "ground-track-field", 
           6 : "bridge", 
           7 : "large-vehicle", 
           8 : "small-vehicle", 
           9 :"helicopter", 
           10 : "swimming-pool", 
           11 : "roundabout", 
           12 : "soccer-ball-field", 
           13 :"plane", 
           14 : "harbor",
           15 : "container-crane"}

In [9]:
def transform_target(original_target_dict_list, output_file, target_size = 1024):
    '''
    Transforms the bounding box coordinates to <label x_center y_center width height> yolo format

    original_target_dict_list: Dictionaries of all bounding boxes in an image
    output_file: Path where new label text file is to be saved
    '''
    calculated_parameters = []
    for bbox_dict in original_target_dict_list:
        label = class_to_idx[bbox_dict['label']]
        if bbox_dict['x2'] >= bbox_dict['x1'] and bbox_dict['y4'] >= bbox_dict['y1']:
            center_x = ((bbox_dict['x1'] + bbox_dict['x2']) / 2) / target_size
            center_y = ((bbox_dict['y1'] + bbox_dict['y4']) / 2) / target_size
            width = (bbox_dict['x2'] - bbox_dict['x1']) / target_size
            height = (bbox_dict['y4'] - bbox_dict['y1']) / target_size
            calculated_parameters.append(f"{label} {center_x} {center_y} {width} {height}")
        else:
            print("Error: Bounding box coordinates wrong.")
            print(output_file)
            break

    if calculated_parameters:
        with open(output_file, 'w') as f:
            for params in calculated_parameters:
                f.write(params + '\n')

In [10]:
def transform_target_for_yolo(original_target_path, output_file_path):
    '''
    Pipeline for transforming targets for yolo

    original_target_path: Path of original DOTA target (format: x1 y1 x2 y2 x3 y3 x4 y4 label difficulty)
    output_file: Path where new yolo label text file is to be saved
    '''
    lines = read_txt(original_target_path)
    extracted_coordinates = extract_hbb(lines)
    transform_target(extracted_coordinates, output_file_path)

In [11]:
if not os.path.exists(join(DATA_DIR, "dota_yolo")):
    os.mkdir(join(DATA_DIR, "dota_yolo"))
    os.mkdir(join(DATA_DIR, "dota_yolo", "train"))
    os.mkdir(join(DATA_DIR, "dota_yolo", "val"))
    os.mkdir(join(DATA_DIR, "dota_yolo", "test"))

In [82]:
# Create train image dir and copy train images
yolo_train_images_dir = join(DATA_DIR, "dota_yolo", "train", "images")
if not os.path.exists(yolo_train_images_dir):
    os.mkdir(yolo_train_images_dir)
    print(f'Created yolo train image directory at {yolo_train_images_dir}')
for image_path in train_images_list:
    base_name = basename(image_path)
    copy_to_path = join(yolo_train_images_dir, base_name)
    shutil.copy(image_path, copy_to_path)
print(f"Copied train images from {train_images_path} to {yolo_train_images_dir}.")

Copied train images from ../04-Data/patches/train_images_patches to ../04-Data/dota_yolo/train/images.


In [83]:
# Create train target dir and transform train targets
yolo_train_target_dir = join(DATA_DIR, "dota_yolo", "train", "labels")
if not os.path.exists(yolo_train_target_dir):
    os.makedirs(yolo_train_target_dir)
    print(f'Created yolo train target directory at {yolo_train_target_dir}')
for original_train_target_path in train_targets_list:
    base_name = basename(original_train_target_path)
    yolo_train_target_path = join(yolo_train_target_dir, base_name)
    transform_target_for_yolo(original_train_target_path, yolo_train_target_path)
print(f"Saved yolo train targets to {yolo_train_target_dir}")

Saved yolo train targets to ../04-Data/dota_yolo/train/labels


In [45]:
# Create val image dir and copy val images
yolo_val_images_dir = join(DATA_DIR, "dota_yolo", "val", "images")
if not os.path.exists(yolo_val_images_dir):
    os.makedirs(yolo_val_images_dir)
    print(f'Created yolo val image directory at {yolo_val_images_dir}')
for image_path in val_images_list:
    base_name = basename(image_path)
    copy_to_path = join(yolo_val_images_dir, base_name)
    shutil.copy(image_path, copy_to_path)
print(f"Copied val images from {val_images_path} to {yolo_val_images_dir}.")

Copied val images from ../04-Data/patches/val_images_patches to ../04-Data/dota_yolo/val/images.


In [46]:
print(val_images_list[0])
print(len(val_images_list))
print(val_targets_list[0])
print(len(val_targets_list))

../04-Data/patches/val_images_patches/P0004/P0004_1024_OL-256_x-0_y-0.png
3139
../04-Data/patches/val_targets_patches/P0004/P0004_1024_OL-256_x-0_y-0.txt
3139


In [47]:
# Create val target dir and transform val targets
yolo_val_target_dir = join(DATA_DIR, "dota_yolo", "val", "labels")
if not os.path.exists(yolo_val_target_dir):
    os.makedirs(yolo_val_target_dir)
    print(f'Created yolo val target directory at {yolo_val_target_dir}')
for original_val_target_path in val_targets_list:
    base_name = basename(original_val_target_path)
    yolo_val_target_path = join(yolo_val_target_dir, base_name)
    transform_target_for_yolo(original_val_target_path, yolo_val_target_path)
print(f"Saved yolo val targets to {yolo_val_target_dir}")

Created yolo val target directory at ../04-Data/dota_yolo/val/labels
Saved yolo val targets to ../04-Data/dota_yolo/val/labels


In [50]:
# Create test image dir and copy test images
yolo_test_images_dir = join(DATA_DIR, "dota_yolo", "test", "images")
if not os.path.exists(yolo_test_images_dir):
    os.makedirs(yolo_test_images_dir)
    print(f'Created yolo test image directory at {yolo_test_images_dir}')
for image_path in test_images_list:
    base_name = basename(image_path)
    copy_to_path = join(yolo_test_images_dir, base_name)
    shutil.copy(image_path, copy_to_path)
print(f"Copied test images from {test_images_path} to {yolo_test_images_dir}.")

Copied test images from ../04-Data/patches/test_images_patches to ../04-Data/dota_yolo/test/images.


In [51]:
# Create test target dir and transform test targets
yolo_test_target_dir = join(DATA_DIR, "dota_yolo", "test", "labels")
if not os.path.exists(yolo_test_target_dir):
    os.makedirs(yolo_test_target_dir)
    print(f'Created yolo test target directory at {yolo_test_target_dir}')
for original_test_target_path in test_targets_list:
    base_name = basename(original_test_target_path)
    yolo_test_target_path = join(yolo_test_target_dir, base_name)
    transform_target_for_yolo(original_test_target_path, yolo_test_target_path)
print(f"Saved yolo test targets to {yolo_test_target_dir}")

Created yolo test target directory at ../04-Data/dota_yolo/test/labels
Saved yolo test targets to ../04-Data/dota_yolo/test/labels


In [82]:
# Create dataset .yaml file
DATA_DIR = "../04-Data"
data = {
    'path': os.path.abspath(DATA_DIR),
    'train': join("train", "images"),
    'val': join("val", "images"),
    'names': {
           0 : "ship", 
           1 : "storage-tank", 
           2 : "baseball-diamond", 
           3 : "tennis-court", 
           4 : "basketball-court", 
           5 : "ground-track-field", 
           6 : "bridge", 
           7 : "large-vehicle", 
           8 : "small-vehicle", 
           9 :"helicopter", 
           10 : "swimming-pool", 
           11 : "roundabout", 
           12 : "soccer-ball-field", 
           13 :"plane", 
           14 : "harbor",
           15 : "container-crane"}
    }

# Writing data to a YAML file
save_path = "dota.yaml"
with open(save_path, 'w') as file:
    yaml.dump(data, file)

print(f"Saved DOTA yaml file to {save_path}.")

#### Create YOLOv8 model from scratch

In [98]:
model = YOLO("yolov8n.yaml")

In [99]:
#results = model.train(data="dota.yaml", epochs=1, imgsz=1024, device="mps", verbose = True)

#### Load trained model

In [17]:
model_bl = YOLO("../best_baseline.pt")
model_os = YOLO("../best_oversampling.pt")
model_aug = YOLO("../best_augment_final.pt")

#### Infer model

In [18]:
def show_prediction(model, images_list, index, ext):
    path_image = images_list[index]
    print(path_image)
    results = model([path_image]) 
    for result in results:
        boxes = result.boxes
        result.show()
        result.save(filename="result" + ext +".png")

In [23]:
index = 1607
show_prediction(model_bl, test_images_list, index, "_bl")
show_prediction(model_os, test_images_list, index, "_os")
show_prediction(model_aug, test_images_list, index, "_aug")

../04-Data/patches_1024/test_images_patches/P1809/P1809_1024_OL-0_x-5120_y-2048.png

0: 1024x1024 108 small-vehicles, 1 plane, 384.6ms
Speed: 4.9ms preprocess, 384.6ms inference, 0.6ms postprocess per image at shape (1, 3, 1024, 1024)
../04-Data/patches_1024/test_images_patches/P1809/P1809_1024_OL-0_x-5120_y-2048.png

0: 1024x1024 1 large-vehicle, 147 small-vehicles, 1 plane, 393.6ms
Speed: 5.7ms preprocess, 393.6ms inference, 0.8ms postprocess per image at shape (1, 3, 1024, 1024)
../04-Data/patches_1024/test_images_patches/P1809/P1809_1024_OL-0_x-5120_y-2048.png

0: 1024x1024 124 small-vehicles, 402.0ms
Speed: 5.0ms preprocess, 402.0ms inference, 0.8ms postprocess per image at shape (1, 3, 1024, 1024)
